In [ ]:
#!pip install --index-url https://test.pypi.org/simple/ comaze
!git clone https://github.com/Near32/comaze-python.git 

In [ ]:
!cd comaze-python; git checkout develop-rl-template; git pull; git status; pip install -e .

In [1]:
import comaze
from comaze.agents import RLCoMaze

class CustomRLCoMaze(RLCoMaze):
  def __init__(self):
    super(CustomRLCoMaze, self).__init__()
  
  def build_agent(self):
    self.agent = None 
  
  def forward(self, obs, reward=0, done=False, infos=None):
    return 44 #'SKIP'

In [2]:
test_agent = CustomRLCoMaze()

In [ ]:
%debug 
options = {}
game_id = test_agent.play_new_game(options=options)

ERROR:root:No traceback has been produced, nothing to debug.


playing existing game {'game_id': '093da619-3945-4850-9618-152d2b396063'}
Joined gameId: 093da619-3945-4850-9618-152d2b396063
Playing as playerId: de09ba52-7f91-471d-9488-eedfcd74c1d0
Waiting for players. (Invite someone: http://teamwork.vs.uni-kassel.de/?gameId=093da619-3945-4850-9618-152d2b396063)
Waiting for players. (Invite someone: http://teamwork.vs.uni-kassel.de/?gameId=093da619-3945-4850-9618-152d2b396063)
Waiting for players. (Invite someone: http://teamwork.vs.uni-kassel.de/?gameId=093da619-3945-4850-9618-152d2b396063)
Waiting for players. (Invite someone: http://teamwork.vs.uni-kassel.de/?gameId=093da619-3945-4850-9618-152d2b396063)
> /home/kevin/Development/git/comaze-python/comaze/agents/rl/rl_comaze.py(167)extract_move_fn()
    166   import ipdb; ipdb.set_trace()
--> 167   move = DiscreteActionMoveExtractor.extract_move(action)
    168 

ipdb> l
    162   player, with the value being a string of uppercase symbol among:
    163   ['Q','W','E','R','T','Y','U','I','O','P']).